# Task 1

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
gpu_start,num_gpus = 9,1
context = Context('removed_redundent1-full-30-no_noise', gpu_start=gpu_start, dep_var=dep_var1, debug=0, mulr=1, fn_feather='tbmData/feather/removed_redundent1-full-30-12.feather')
stat_x = context.stat_x
valid_size = int(context.num_cycles * context.valid_ratio * context.mulr)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=valid_size)

In [ ]:
hyper_params = {
    'n_hidden': 200,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.6,
    'weight_p': 0.5,
    'layers': [3*context.n_cont, 5, len(context.dep_var)],
    'drops': [0.2] * 3,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat(),
#     'loss_func': l1
}

learner = get_cont_model(context, databunch, hyper_params)

In [ ]:
learner.load('200_final-task-1_2019-03-15_18:28:17');

In [ ]:
test1_fns = sorted(Path('tbmData/test/附件1：必选课题一测试数据集').glob('*.mat'))
column_names = pd.read_csv('tbmData/data/CREC188_20160809.txt', delimiter='\t', nrows=1).columns # get column names from training data

In [ ]:
def read_test_mat_as_df(mat_fn, columns):
    test_input = scipy.io.loadmat(mat_fn)['a']
    test_df = pd.DataFrame(test_input[:,:-1], columns=columns[2:])
    return test_df

save_dir = Path('tbmData/test/test1_feathers')
for mat_fn in test1_fns:
    df = read_test_mat_as_df(mat_fn, column_names)
    save_feather_fn = save_dir/('cycle' + mat_fn.stem + '.feather')
    df.to_feather(str(save_feather_fn))

context = Context('save-after-removing-redundent-columns', fn_cycles='tbmData/test/test1_feathers', debug=0, mulr=1, load_data=False)
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)

drop redundant columns, which should drop zeroed columns as well

In [ ]:
dropped = np.load('tbmData/cycles-removed-redundent-columns/dropped.npy').tolist()

fmtr.cycles.drop(columns=dropped+['桩号']+['EP2次数设置', '刀盘喷水增压泵压力'], inplace=True, errors='ignore')
# , '内循环水罐液位', '拖拉油缸最小允许压力设置'

cycles = fmtr.cycles

In [ ]:
len(dropped)

normalize

In [ ]:
normalized_x = normalize_a(cycles, *stat_x)

flatten

In [ ]:
xb = [tensor(normalized_x.loc[i].values.flatten())[None].float() for i in normalized_x.index.levels[0]]

In [ ]:
xb = torch.cat(xb)
xb.shape, xb.dtype

In [ ]:
model = learner.model
model.eval()
input = (torch.zeros(len(xb), dtype=torch.float32).cuda(), xb.cuda())
pb = model(*input)

In [ ]:
pb = denormalize(pb[0], *context.stat_y)
pb = to_np(pb)

In [ ]:
output_template = 'tbmData/test/附件3：模型输出结果保存文件/Result_test_1-50.csv'
template = pd.read_csv(output_template,encoding='gbk')

In [ ]:
template.iloc[:,1:3] = pb

In [ ]:
submit_dir = Path('submissions/submission2')
submit_dir.mkdir(parents=True,exist_ok=True)
template.to_csv(str(submit_dir/'Result_test_1-50.csv'), float_format='%.1f',index=None, encoding='gbk')